In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from scipy import interp
import math
from tensorflow import contrib
tfe = contrib.eager
import tensorflow.python.saved_model
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

from sklearn.model_selection import KFold   

#from triplet_loss import batch_all_triplet_loss
#from triplet_loss import batch_hard_triplet_loss

import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

print(tf.__version__)

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

from matplotlib.pyplot import figure
import itertools

In [ ]:
face_cascade = cv2.CascadeClassifier('xml_files/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('xml_files/haarcascade_eye.xml')

Training_Data = r'Chokepoint/jpg'
Testing_Data = r'Chokepoint/Video'

IMG_SIZE = 96
LearningRate = math.pow(10, -4) * 1
print(LearningRate)
batch_size = 37
dropout=0.8
peopleNum = 21
hm_epochs = 10000
file_path = os.getcwd()+'\model_checkpoints\model12.ckpt'

In [ ]:
def img_cropper(img_path):    
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        print(faces)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        cv2.imshow('img',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return img

def image_labeler(img):
    labelName = img.split('.')[-2]
    labelName = labelName.split('-')[-1]
    if labelName == 'person1': return [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person3': return [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person4': return [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person5': return [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person6': return [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person7': return [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person9': return [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person10': return [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person11': return [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person12': return [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person13': return [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person14': return [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif labelName == 'person15': return [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif labelName == 'person16': return [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif labelName == 'person17': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
    elif labelName == 'person18': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif labelName == 'person19': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif labelName == 'person20': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif labelName == 'person21': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif labelName == 'person22': return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    else: return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
    
def image_labeler_converter(labelName):
    if np.array_equal(labelName,[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 0
    elif np.array_equal(labelName,[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 1
    elif np.array_equal(labelName,[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 2
    elif np.array_equal(labelName,[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 3
    elif np.array_equal(labelName,[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 4
    elif np.array_equal(labelName,[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 5
    elif np.array_equal(labelName,[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 6
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]): return 7
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]): return 8
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]): return 9
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]): return 10
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]): return 11
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]): return 12
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]): return 13
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]): return 14
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]): return 15
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]): return 16
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]): return 17
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]): return 18
    elif np.array_equal(labelName,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]): return 19
    else: return 20


def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(Testing_Data)):
        path = os.path.join(Testing_Data,img)
        img_num = img_num.split('.')[-2]
        img_num = img_num.split('-')[-2]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])

    print(testing_data)
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

def process_images_training_other():
    training_data = []
    for img in tqdm(os.listdir(Training_Data)):
        label = image_labeler(img)
        path = os.path.join(Training_Data,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])  

    shuffle(training_data)
    np.save('training_data/training_data.npy', training_data)
    return training_data

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1], strides=[1,1,1,1],padding='SAME')

def conv_layer(input, shape, bias):
    W = weight_variable(shape)
    b = bias_variable(bias)
    return tf.nn.relu(conv2d(input, W)+b)

def full_layer(input, size, bias):
    W = weight_variable(size)
    b = bias_variable(bias)
    return tf.nn.relu(tf.matmul(input, W)+b)

def inception_layer(input,input_size,output_size,mid_output):
    conv1_1x1 = conv_layer(input,shape = [1,1,input_size,output_size], bias = [output_size])
    conv2_1x1 = conv_layer(input,shape = [1,1,input_size,mid_output], bias = [mid_output])
    conv3_1x1 = conv_layer(input,shape = [1,1,input_size,mid_output], bias = [mid_output])
    maxpool4_3x3 = max_pool_3x3(input)
    
    conv2_3x3 = conv_layer(conv2_1x1, shape=[3,3,mid_output,output_size],bias = [output_size])
    conv3_5x5 = conv_layer(conv3_1x1, shape=[5,5,mid_output,output_size],bias = [output_size])
    conv4_1x1 = conv_layer(maxpool4_3x3,shape = [1,1,input_size,output_size], bias = [output_size])
    
    inception = tf.nn.relu(tf.nn.bias_add(tf.concat([conv1_1x1,conv2_3x3,conv3_5x5,conv4_1x1],3),bias_variable([output_size*4])))
    
    return inception

people = ['person1','person3','person4','person5','person6','person7','person9','person10','person11','person12','person13','person14','person15','person16','person17','person18','person19','person20','person21','person22','unknown',]

In [ ]:
train_data = process_images_training_other()

In [ ]:
train = train_data[:-333]
test = train_data[-333:]

In [ ]:
def true_model(data):
    #tf.reset_default_graph()
    data = tf.reshape(data, shape = [-1,96,96,1])

    conv1 = conv_layer(data,shape = [5,5,1,64], bias = [64])
    conv1_pool = max_pool_2x2(conv1)
    conv2 = conv_layer(conv1_pool, shape=[5,5,64,128],bias = [128])
    conv2_pool = max_pool_2x2(conv2)
    conv3 = conv_layer(conv2_pool, shape=[5,5,128,256],bias = [256])
    conv3_pool = max_pool_2x2(conv3)
    conv4 = conv_layer(conv3_pool, shape=[5,5,256,256],bias = [256])
    conv4_pool = max_pool_2x2(conv4)
    conv5 = conv_layer(conv4_pool, shape=[5,5,256,256],bias = [256])
    conv5_pool = max_pool_2x2(conv5)
    conv6 = conv_layer(conv5_pool, shape=[5,5,256,256],bias = [256])
    conv6_pool = max_pool_2x2(conv6)
    
    inception1 = inception_layer(conv6_pool,256,128,256)
    inception2 = inception_layer(inception1,512,256,128)
    inception3 = inception_layer(inception2,1024,64,256)
    inception4 = inception_layer(inception3,256,128,64)
    inception5 = inception_layer(inception4,512,128,64)
    inception6 = inception_layer(inception5,512,256,128)
    
    inception_pool = max_pool_2x2(inception6)
    
    conv6_flat = tf.reshape(inception_pool, [-1,1*1*256*4])
  
    fc_1= full_layer(conv6_flat, [1*1*256*4, 2048], [2048])
    fc_1 = tf.nn.dropout(fc_1, dropout)
    fc_2= full_layer(fc_1, [2048, 2048], [2048])
    fc_2 = tf.nn.dropout(fc_2, dropout)
    output = tf.matmul(fc_2, weight_variable([2048, peopleNum])) + bias_variable([peopleNum])
    print(output)
    return output

def training_CNN_triplet(x):   
    prediction = true_model(x)
    tf.identity(prediction, name="Output_ph")
    cost = tf.reduce_mean( tf.losses.softmax_cross_entropy(logits=prediction, onehot_labels= y_) )
    optimizer = tf.train.AdamOptimizer(LearningRate).minimize(cost)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))       
    loss_list = []
    training_acc_list = []
    test_acc_list = []

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i = 0
            while i < len(train):
                start = i 
                end = i + batch_size

                batch_x = np.array(X_data[start:end])
                batch_y = np.array(Y_labels[start:end])

                _, c = sess.run([optimizer,cost], feed_dict={x: batch_x, y_: batch_y})
                epoch_loss += c
                i+= batch_size
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            loss_list.append(epoch_loss)
            if (epoch+1)%1 == 0:
                training_acc_list.append(training_acc_CNN(sess, accuracy))
                test_acc_list.append(validation_acc_CNN(sess, accuracy))

        predictions = prediction.eval(feed_dict={x: cX_data}, session=sess)
        
        
        plt.plot(loss_list)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title("Loss History")
        plt.show()
        plt.plot(training_acc_list)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title("Training Accuracy")
        plt.show()
        plt.plot(test_acc_list)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title("Testing Accuracy")
        plt.show()
        roc_auc_graph("Testing",predictions,cY_labels) 
        confusion_matrix_make(sess,predictions,cY_labels)

        precision_recall(sess,predictions,cY_labels)
        perf_measure(cY_labels, predictions)
        
        live_prediction(sess,prediction)
        
        print('\nSaving...')
        inputs_dict = {
            "features_data_ph": x,
            "labels_data_ph": y_
        }
        outputs_dict = {
            "Output_ph": prediction
        }
        #tf.saved_model.simple_save(sess, file_path, inputs_dict, outputs_dict)
        print("Done")

In [ ]:
def training_acc_CNN(sess, accuracy):     
    training_accuracy_results = []
    cls_pred = np.zeros(shape=len(test), dtype=np.int)
    i = 0
    while i < len(test):
        start = i 
        end = i + batch_size

        batch_x = np.array(X_data[start:end])
        batch_y = np.array(Y_labels[start:end])

        training_accuracy_results.append(sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y})) 
        i+= batch_size
        
    acc = np.mean(training_accuracy_results)
    print("Training Accuracy: {}".format(acc *100))
    return acc
def validation_acc_CNN(sess, accuracy):     
    validation_accuracy_results = []
    cls_pred = np.zeros(shape=len(test), dtype=np.int)
    i = 0
    while i < len(test):
        start = i 
        end = i + batch_size

        batch_x = np.array(cX_data[start:end])
        batch_y = np.array(cY_labels[start:end])

        validation_accuracy_results.append(sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y})) 
        i+= batch_size
        
    acc = np.mean(validation_accuracy_results)
    print("Validation Accuracy: {}".format(acc*100))
    return acc
    
def roc_auc_graph(typ,predictions,labels):
    test = []
    for i in range(len(labels)):
        test.append(image_labeler_converter(labels[i]))
    test = label_binarize(test, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    fpr, tpr, _ = roc_curve(test.ravel(), predictions.ravel())
    roc_auc = auc(fpr, tpr)
    
    print('roc_auc: {}'.format(roc_auc))
    figure(num=None, figsize=(4, 3), dpi=80, facecolor='w', edgecolor='k')
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('{} Receiver operating characteristic'.format(typ))
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(cm,names, title='Confusion matrix', cmap=plt.cm.Blues,normalize=False):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=90)
    plt.yticks(tick_marks, names)
    plt.tight_layout()


    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
def confusion_matrix_make(typ,prediction,labels):
    test = []
    for i in range(len(labels)):
        test.append(image_labeler_converter(labels[i]))
    test = label_binarize(test, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    y_test2 = np.argmax(test,axis=1)
    pred = np.argmax(prediction,axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_test2, pred, labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    np.set_printoptions(precision=2)
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print('Normalized confusion matrix')

    figure(num=None, figsize=(4,4), dpi=80, facecolor='w', edgecolor='k')
    plot_confusion_matrix(cm_normalized, people, title='{} Normalized confusion matrix'.format(typ), normalize = True)

    plt.show()
        
def precision_recall(typ,predictions,labels):
    test = []
    for i in range(len(labels)):
        test.append(image_labeler_converter(labels[i]))
    test = label_binarize(test, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    precision, recall, _ = precision_recall_curve(test.ravel(), predictions.ravel())
    
    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    
    
    average_precision = average_precision_score(test.ravel(), predictions.ravel())
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision))
    
    figure(num=None, figsize=(4, 3), dpi=80, facecolor='w', edgecolor='k')
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('{typ} Average precision score, micro-averaged over all classes: AP={average_precision}'
        .format(average_precision=average_precision,typ=typ))
    
def perf_measure(labels, prediction):
    test = []
    for i in range(len(labels)):
        test.append(image_labeler_converter(labels[i]))
    test = label_binarize(test, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    y_test2 = np.argmax(test,axis=1)
    pred = np.argmax(prediction,axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_test2, pred, labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    print('tp: {}'.format(TP))
    print('fp: {}'.format(FP))
    print('tN: {}'.format(TN))
    print('fn: {}'.format(FN)) 
    
    
def live_prediction(sess,prediction):
    fig=plt.figure()

    for num,data in enumerate(test[:12]):
        img_num = data[1]
        img_data = data[0]

        y = fig.add_subplot(3,4,num+1)
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        
        model_out = prediction.eval(feed_dict={x: [data]}, session=sess)
        if np.argmax(model_out) == 0: str_label='person1'
        elif np.argmax(model_out) == 1: str_label='person3'
        elif np.argmax(model_out) == 2: str_label='person4'
        elif np.argmax(model_out) == 3: str_label='person5'
        elif np.argmax(model_out) == 4: str_label='person6'
        elif np.argmax(model_out) == 5: str_label='person7'
        elif np.argmax(model_out) == 6: str_label='person9'
        elif np.argmax(model_out) == 7: str_label='person10'
        elif np.argmax(model_out) == 8: str_label='person11'
        elif np.argmax(model_out) == 9: str_label='person12'
        elif np.argmax(model_out) == 10: str_label='person13'
        elif np.argmax(model_out) == 11: str_label='person14'
        elif np.argmax(model_out) == 12: str_label='person15'
        elif np.argmax(model_out) == 13: str_label='person16'
        elif np.argmax(model_out) == 14: str_label='person17'
        elif np.argmax(model_out) == 15: str_label='person18'
        elif np.argmax(model_out) == 16: str_label='person19'
        elif np.argmax(model_out) == 17: str_label='person20'
        elif np.argmax(model_out) == 18: str_label='person21'
        elif np.argmax(model_out) == 19: str_label='person22'
        else: str_label='unknown'

        y.imshow(orig,cmap='gray')
        plt.title(str_label)
        y.axes.get_xaxis().set_visible(False)
        y.axes.get_yaxis().set_visible(False)
    plt.show()
            

In [ ]:
x = tf.placeholder(tf.float32, shape=[None,96,96,1], name='placeholder_input')
y_ = tf.placeholder(tf.float32, shape=[None, peopleNum], name='placeholder_labels') 

y_true_cls = tf.argmax(y_, axis=1)

X_data = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE,IMG_SIZE, 1)   
Y_labels = [i[1] for i in train]

cX_data = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE,IMG_SIZE, 1)   
cY_labels = [i[1] for i in test]



In [ ]:
training_CNN_triplet(x)